# Langchain + GPT-3

Quick overview how to build systems that use GPT-3 and langchain.

    Building applications with LLMs through composability

https://langchain.readthedocs.io/en/latest/index.html

https://beta.openai.com

In [ ]:
import os
import sqlite3

from langchain.agents import ZeroShotAgent, Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains.llm import LLMChain
from langchain.chains.llm_math.base import LLMMathChain
from langchain.chains.sql_database.base import SQLDatabaseChain
from langchain.llms.openai import OpenAI
from langchain.serpapi import SerpAPIChain
from langchain.sql_database import SQLDatabase

from dotenv import load_dotenv

load_dotenv()

DB_NAME = "bookmarks.db"

### Bootstrap

In [ ]:
if os.path.exists(DB_NAME):
    os.remove(DB_NAME)

# create a new database
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()
# create bookmarks table
cursor.execute(
    """
    CREATE TABLE bookmarks (
        id integer primary key autoincrement,
        name text,
        url text
    )
"""
)

# insert dummy records
for i in range(20):
    cursor.execute(
        """
        INSERT INTO bookmarks (name, url)
        VALUES (?, ?)
    """,
        (f"bookmark {i}", f"http://bookmark.com/{i}/"),
    )

# save changes
conn.commit()

# close connection
conn.close()

### Models

In [ ]:
llm = OpenAI(temperature=0, model_name="text-davinci-003")
db = SQLDatabase.from_uri(f"sqlite:///{DB_NAME}")

### Chains

In [ ]:
search = SerpAPIChain()
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)
math_chain = LLMMathChain(llm=llm, verbose=True)

### Tools

In [ ]:
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or search the internet."
    ),
    Tool(
        name="Bookmarks DB",
        func=db_chain.run,
        description="useful for when you need to answer questions about bookmarks. Input should be in the form of a question or statement"
    ),
    Tool(
        name="Calculator",
        func=math_chain.run,
        description="useful for when you need to answer questions about math and calculate"
    ),
]

### Prompt

In [ ]:
prefix = """Have a conversation with a human, answering the following questions as best you can and a friendly and specific manner. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "chat_history"]
)
memory = ConversationBufferMemory(memory_key="chat_history")


### Agent

In [ ]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt, memory=memory)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)

In [ ]:
agent.run("how many bookmarks are there?")

In [ ]:
agent.run("Add a bookmark for Apple")

In [ ]:
agent.run("How many bookmarks begin with 'b'?")

In [ ]:
agent.run("what's the URL of the wikipedia page for Apple?")

In [ ]:
agent.run("Ok add that page to my bookmarks as well")

In [ ]:
agent.run("update that bookmark to 'Apple Computer Wiki Page'")

### Inspect memory

In [ ]:
import pprint
pprint.pprint(memory.buffer.splitlines(), width=120)

### Calculations via Python REPL

In [ ]:
memory.clear()
#agent.run("I have a box 1.5 meters by 2.5 meters. What is the volume?")
agent.run("How many of the integers between 0 and 99 inclusive are divisible by 8?")

In [ ]:
memory.buffer = ""
agent.run("I have a ball 1m in diameter. What is its volume in cubic meters?")

In [ ]:
agent.run("Thanks. Round that for me")

### Bonus: World Cup

In [ ]:
memory.clear()
agent.run("What was the final score in the match between Portugal vs Switzerland in the 2022 World Cup?")

In [ ]:
agent.run("Give me 10 facts about the winning country")